In [132]:
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: ...working... done

# All requested packages already installed.



In [160]:
import requests
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
import json
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium 

print('Libraries imported.')

Libraries imported.


In [161]:
wikiurl="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
table_class="wikitable sortable jquery-tablesorter"
response=requests.get(wikiurl)
print(response.status_code)

200


In [162]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(response.text, 'html.parser')
indiatable=soup.find('table',{'class':"wikitable"})

In [163]:
df=pd.read_html(str(indiatable))
# convert list to dataframe
df=pd.DataFrame(df[0])
print(df.head())

  Postal Code           Borough              Neighbourhood
0         M1A      Not assigned               Not assigned
1         M2A      Not assigned               Not assigned
2         M3A        North York                  Parkwoods
3         M4A        North York           Victoria Village
4         M5A  Downtown Toronto  Regent Park, Harbourfront


In [164]:
df=df[df.Borough != 'Not assigned']
df

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [165]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 103 entries, 2 to 178
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Postal Code    103 non-null    object
 1   Borough        103 non-null    object
 2   Neighbourhood  103 non-null    object
dtypes: object(3)
memory usage: 3.2+ KB


In [166]:
response = requests.get("https://api.mapbox.com/geocoding/v5/mapbox.places/M5A.json?types=postcode&country=CA&access_token=pk.eyJ1IjoiYWF5dXNoMjQwIiwiYSI6ImNraW9vNGY4ejA1NmwyeXBhNzY3Mmg4OWoifQ.NvFrqSGBtGzrbhRTTsoyvA")
a=response.json()
def jprint(obj):
    # create a formatted string of the Python JSON object
    text = json.dumps(obj, sort_keys=True, indent=4)
    print(text)
b=a["features"][0]["center"]
long=b[1]
lat=b[0]
print(long)
print(lat)

43.65
-79.36


In [169]:
Longitude=[]
Latitude=[]
for i in df["Postal Code"]:
    response = requests.get("https://api.mapbox.com/geocoding/v5/mapbox.places/{}.json?types=postcode&country=CA&access_token=pk.eyJ1IjoiYWF5dXNoMjQwIiwiYSI6ImNraW9vNGY4ejA1NmwyeXBhNzY3Mmg4OWoifQ.NvFrqSGBtGzrbhRTTsoyvA".format(i))
    a=response.json()
    def jprint(obj):
        # create a formatted string of the Python JSON object
        text = json.dumps(obj, sort_keys=True, indent=4)
        print(text)
    b=a["features"][0]["center"]
    long=b[0]
    Longitude.append(long)
    lat=b[1]
    Latitude.append(lat)
df["Longitude"] = Longitude
df["Latitude"] = Latitude
df.head()

,Postal Code,Borough,Neighbourhood,Longitude,Latitude
2,M3A,North York,Parkwoods,-79.337754,43.764792
3,M4A,North York,Victoria Village,-79.319071,43.725724
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",-79.360000,43.650000
5,M6A,North York,"Lawrence Manor, Lawrence Heights",-79.453165,43.727181
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",-79.392518,43.664611


In [170]:
df

,Postal Code,Borough,Neighbourhood,Longitude,Latitude
2,M3A,North York,Parkwoods,-79.337754,43.764792
3,M4A,North York,Victoria Village,-79.319071,43.725724
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",-79.360000,43.650000
5,M6A,North York,"Lawrence Manor, Lawrence Heights",-79.453165,43.727181
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",-79.392518,43.664611
...,...,...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",-79.524390,43.645349
165,M4Y,Downtown Toronto,Church and Wellesley,-79.386177,43.669757
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",-79.322720,43.662910
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",-79.488215,43.627693


In [172]:
longitude=-79.36
latitude=43.65
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
for i in range(0,103):
    map_toronto.add_child(folium.features.CircleMarker([Latitude[i],Longitude[i]],radius=5,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False))
map_toronto
